In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

2023-02-09 14:54:06.526887: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-09 14:54:06.540779: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [59]:
import pandas as pd
data = pd.read_csv("/home/mike/Downloads/spam detection/SPAM text message 20170820 - Data.csv")

In [60]:
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [61]:
y = data.Category

In [62]:
y

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: Category, Length: 5572, dtype: object

In [63]:
emails = tf.data.TextLineDataset(data.Message)

In [47]:
emails

<TextLineDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [48]:
# emails = tf.data.TextLineDataset("/home/mike/Downloads/spam detection/SPAM text message 20170820 - Data.csv")

In [49]:
# batch_size = 1024
# seed = 123
# train_ds = tf.keras.utils.text_dataset_from_directory(
#     "/home/mike/Downloads/spam detection/SPAM text message 20170820 - Data.csv", batch_size=batch_size, validation_split=0.2,
#     subset='training', seed=seed)
# val_ds = tf.keras.utils.text_dataset_from_directory(
#     "/home/mike/Downloads/spam detection/SPAM text message 20170820 - Data.csv", batch_size=batch_size, validation_split=0.2,
#     subset='validation', seed=seed)

In [51]:
# data = tf.data.Dataset.zip((emails, y))

In [52]:
AUTOTUNE = tf.data.AUTOTUNE

data = emails.batch(1024).cache().prefetch(buffer_size=AUTOTUNE)


In [53]:
# for line in data.take(10):
#     print(line.numpy())

In [54]:
embedding_layer = tf.keras.layers.Embedding(1000, 5)

In [55]:
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[-0.00099806, -0.0012598 , -0.02478626, -0.00018432,  0.02154752],
       [-0.00849898,  0.0328276 , -0.03346795,  0.04410434,  0.0107677 ],
       [ 0.04955406,  0.0321137 ,  0.01350272, -0.02365598,  0.01528664]],
      dtype=float32)

In [64]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = emails.map(lambda x: x)
# vectorize_layer.adapt(text_ds)

In [65]:
vectorize_layer.adapt(text_ds)

NotFoundError: Graph execution error:

Detected at node 'IteratorGetNext' defined at (most recent call last):
    File "/home/mike/anaconda3/envs/tensorflow/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/mike/anaconda3/envs/tensorflow/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/mike/anaconda3/envs/tensorflow/lib/python3.10/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/mike/anaconda3/envs/tensorflow/lib/python3.10/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/home/mike/anaconda3/envs/tensorflow/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/mike/anaconda3/envs/tensorflow/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_